In [1]:
# using gini index,pre pruning and only a few thresholds for best split.

import csv
from google.colab import files
import numpy as np
import sys


# Enter You Name Here
myname = "Amulya" #Amar-Akbar-Antony"

# Implement your decision tree below

class DecisionTree():

    def gini(self,set):# gini index calculation
        p0=p1=0
        for r in set:
            if float(r[-1])==0:
                p0=p0+1
            elif float(r[-1])==1:
                p1=p1+1
        if p0!=0 and p1!=0:
            return 1-(p1/(p0+p1))**2-(p0/(p0+p1))**2
        else:
            return 0
    def gin(self,parent,c1,c2):# gini impurity
        return (self.gini(c1))*len(c1)/len(parent)+(self.gini(c2))*len(c2)/len(parent)
    def split_left(self,thresh,index,data_set):#left child
        left=[]
        for parent in data_set:
            if float(parent[index])<float(thresh):
                left.append(parent)
        return left
    def split_right(self,thresh,index,data_set):#right child
        right=[]
        for parent in data_set:
            if float(parent[index])>=float(thresh):
                right.append(parent)
        return right

    def best_split(self,training_set):#selecting best value to split on using gini impurity and using 20 values between the minimum and maximum possible values.
        min_gin=(np.inf)
        corres_ind=0
        thresh_max=0      
        for i in range(0,len(training_set[0])-1):

            
            thresh=[]
            for j in range(0,len(training_set)):
                thresh.append(float(training_set[j][i]))
            features=np.linspace(min(thresh),max(thresh),30)
            
            for l in range(0,len(features)):
                
                left=self.split_left(float(features[l]),i,training_set)
                right=self.split_right(float(features[l]),i,training_set)
                gin=self.gin(training_set,left,right)
                if float(gin)<float(min_gin):
                    thresh_max=features[l]
                    corres_ind=i

                    min_gin=gin

        return [thresh_max,corres_ind]
            

    tree = {}

    def learn(self, training_set):
        # implement this function
        self.tree = {}
        if self.gini(training_set)!=0 :
            if len(training_set)<=10:#converting into leaf node before the node becomes pure.
              #Pre pruning the decision tree. I observed that the average number of samples in each leaf nodes
              #was around 10. So I decided to use that as the stopping condition .
                
                self.tree['right']=None
                self.tree['left']=None
                self.tree['index']=None
                self.tree['thresh_val']=None
                c0=0
                c1=0
                for instance in training_set:
                    if float(instance[-1])==0:
                        c0=c0+1
                    elif float(instance[-1])==1:
                        c1=c1+1
            #print(c0,c1)
                if c0>=c1:
                    self.tree['value']=0
                elif c1>c0:
                    self.tree['value']=1
                return self.tree        
            self.tree['thresh_val']=(self.best_split(training_set))[0]
            self.tree['index']=(self.best_split(training_set))[1]
            thre=self.tree['thresh_val']
            ind=self.tree['index']
            left_subtree=DecisionTree()#recursively growing the tree
            right_subtree=DecisionTree()
            left=self.split_left(thre,ind,training_set)
            right=self.split_right(thre,ind,training_set)
            left_subtree.learn(left)
            right_subtree.learn(right)
            self.tree['left']=left_subtree.tree
            self.tree['right']=right_subtree.tree
            self.tree['value']=None
        else: # if the node is pure, it will be converted into a leaf node
            self.tree['right']=None
            self.tree['left']=None
            self.tree['index']=None
            self.tree['thresh_val']=None
            c0=0
            c1=0
            for instance in training_set:
                #print(instance)
                
                
                if float(instance[-1])==0:
                    
                    c0=c0+1
                elif float(instance[-1])==1:
                    c1=c1+1
            #print(c0,c1)
            if c0>=c1:
                self.tree['value']=0
            elif c1>c0:
                self.tree['value']=1
        return self.tree        


    # implement this function
    def classify(self, test_instance,x):# we find the value in the dictionary recursively
        #result = 0 # baseline: always classifies as 0
        if (x['value']!=None): 
            
             
            return x['value']
        else:
            
            value_rec=test_instance[x['index']]
            if float(value_rec)<float(x['thresh_val']):
                #print(float(value_rec),float(x['thresh_val']))
                #print(x['left'])
                return self.classify(test_instance,x['left'])
            elif float(value_rec)>=float(x['thresh_val']):
                return self.classify(test_instance,x['right'])

        #return result

def run_decision_tree():
    f=files.upload()
    # Load data set
    with open("wine-dataset.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print("Number of records: %d" % len(data))

    # Split training/test sets
    # You need to modify the following code for cross validation.
    accuracies=[]
    for K in range(0,10):
        
        training_set = [x for i, x in enumerate(data) if i % 10 != K]
        test_set = [x for i, x in enumerate(data) if i % 10== K]
    
        tree = DecisionTree()
        
    # Construct a tree using training set
        dic=tree.learn( training_set)
        #print(dic)
    #print(dic['left']['left']['left']['value'])

    # Classify the test set using the tree we just constructed
        results = []
        for instance in test_set:
            result = tree.classify( instance,dic )
            #print(result)
            results.append( result == float(instance[-1]))
        

    # Accuracy
        accuracy = float(results.count(True))/float(len(results))
        print("accuracy: %.4f" % accuracy)
        accuracies.append(accuracy)       
    sum=0
    for i in range(0,len(accuracies)):
        sum=float(accuracies[i])+sum
    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"result.txt", "w")
    g=sum/len(accuracies)
    f.write("avg. accuracy: %.4f" %g)
    f.close()


if __name__ == "__main__":
    run_decision_tree()

Saving wine-dataset.csv to wine-dataset.csv
Number of records: 4898
accuracy: 0.8408
accuracy: 0.8510
accuracy: 0.8286
accuracy: 0.8184
accuracy: 0.8245
accuracy: 0.7837
accuracy: 0.8469
accuracy: 0.8286
accuracy: 0.8262
accuracy: 0.8364


In [1]:
# using gini index and only a few thresholds for best split.

import csv
from google.colab import files
import numpy as np
import sys


# Enter You Name Here
myname = "Amulya" #Amar-Akbar-Antony"

# Implement your decision tree below

class DecisionTree():

    def gini(self,set):# gini index calculation
        p0=p1=0
        for r in set:
            if float(r[-1])==0:
                p0=p0+1
            elif float(r[-1])==1:
                p1=p1+1
        if p0!=0 and p1!=0:
            return 1-(p1/(p0+p1))**2-(p0/(p0+p1))**2
        else:
            return 0
    def gin(self,parent,c1,c2):# gini impurity
        return (self.gini(c1))*len(c1)/len(parent)+(self.gini(c2))*len(c2)/len(parent)
    def split_left(self,thresh,index,data_set):#left child
        left=[]
        for parent in data_set:
            if float(parent[index])<float(thresh):
                left.append(parent)
        return left
    def split_right(self,thresh,index,data_set):#right child
        right=[]
        for parent in data_set:
            if float(parent[index])>=float(thresh):
                right.append(parent)
        return right

    def best_split(self,training_set):#selecting best value to split on using gini impurity and using 20 values between the minimum and maximum possible values.
        min_gin=(np.inf)
        corres_ind=0
        thresh_max=0      
        for i in range(0,len(training_set[0])-1):

            
            thresh=[]
            for j in range(0,len(training_set)):
                thresh.append(float(training_set[j][i]))
            features=np.linspace(min(thresh),max(thresh),30)
            
            for l in range(0,len(features)):
                
                left=self.split_left(float(features[l]),i,training_set)
                right=self.split_right(float(features[l]),i,training_set)
                gin=self.gin(training_set,left,right)
                if float(gin)<float(min_gin):
                    thresh_max=features[l]
                    corres_ind=i

                    min_gin=gin

        return [thresh_max,corres_ind]
            

    tree = {}

    def learn(self, training_set):
        # implement this function
        self.tree = {}
        if self.gini(training_set)!=0 :
            #print(c0,c1)
     
            self.tree['thresh_val']=(self.best_split(training_set))[0]
            self.tree['index']=(self.best_split(training_set))[1]
            thre=self.tree['thresh_val']
            ind=self.tree['index']
            left_subtree=DecisionTree()#recursively growing the tree
            right_subtree=DecisionTree()
            left=self.split_left(thre,ind,training_set)
            right=self.split_right(thre,ind,training_set)
            left_subtree.learn(left)
            right_subtree.learn(right)
            self.tree['left']=left_subtree.tree
            self.tree['right']=right_subtree.tree
            self.tree['value']=None
        else: # if the node is pure, it will be converted into a leaf node
            self.tree['right']=None
            self.tree['left']=None
            self.tree['index']=None
            self.tree['thresh_val']=None
            c0=0
            c1=0
            for instance in training_set:
                #print(instance)
                
                
                if float(instance[-1])==0:
                    
                    c0=c0+1
                elif float(instance[-1])==1:
                    c1=c1+1
            #print(c0,c1)
            if c0>=c1:
                self.tree['value']=0
            elif c1>c0:
                self.tree['value']=1
        return self.tree        


    # implement this function
    def classify(self, test_instance,x):# we find the value in the dictionary recursively
        #result = 0 # baseline: always classifies as 0
        if (x['value']!=None): 
            
             
            return x['value']
        else:
            
            value_rec=test_instance[x['index']]
            if float(value_rec)<float(x['thresh_val']):
                #print(float(value_rec),float(x['thresh_val']))
                #print(x['left'])
                return self.classify(test_instance,x['left'])
            elif float(value_rec)>=float(x['thresh_val']):
                return self.classify(test_instance,x['right'])

        #return result

def run_decision_tree():
    f=files.upload()
    # Load data set
    with open("wine-dataset.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print("Number of records: %d" % len(data))

    # Split training/test sets
    # You need to modify the following code for cross validation.
    accuracies=[]
    for K in range(0,10):
        
        training_set = [x for i, x in enumerate(data) if i % 10 != K]
        test_set = [x for i, x in enumerate(data) if i % 10== K]
    
        tree = DecisionTree()
        
    # Construct a tree using training set
        dic=tree.learn( training_set)
        #print(dic)
    #print(dic['left']['left']['left']['value'])

    # Classify the test set using the tree we just constructed
        results = []
        for instance in test_set:
            result = tree.classify( instance,dic )
            #print(result)
            results.append( result == float(instance[-1]))
        

    # Accuracy
        accuracy = float(results.count(True))/float(len(results))
        print("accuracy: %.4f" % accuracy)
        accuracies.append(accuracy)       
    sum=0
    for i in range(0,len(accuracies)):
        sum=float(accuracies[i])+sum
    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"result.txt", "w")
    g=sum/len(accuracies)
    f.write("avg. accuracy: %.4f" %g)
    f.close()


if __name__ == "__main__":
    run_decision_tree()

Saving wine-dataset.csv to wine-dataset.csv
Number of records: 4898
accuracy: 0.8551
accuracy: 0.8694
accuracy: 0.8388
accuracy: 0.8061
accuracy: 0.8469
accuracy: 0.7878
accuracy: 0.8449
accuracy: 0.8429
accuracy: 0.8364
accuracy: 0.8282


In [2]:
# using gini index,pre pruning and only a few thresholds for best split.

import csv
from google.colab import files
import numpy as np
import sys


# Enter You Name Here
myname = "Amulya" #Amar-Akbar-Antony"

# Implement your decision tree below

class DecisionTree():

    def gini(self,set):# gini index calculation
        p0=p1=0
        for r in set:
            if float(r[-1])==0:
                p0=p0+1
            elif float(r[-1])==1:
                p1=p1+1
        if p0!=0 and p1!=0:
            return 1-(p1/(p0+p1))**2-(p0/(p0+p1))**2
        else:
            return 0
    def gin(self,parent,c1,c2):# gini impurity
        return (self.gini(c1))*len(c1)/len(parent)+(self.gini(c2))*len(c2)/len(parent)
    def split_left(self,thresh,index,data_set):#left child
        left=[]
        for parent in data_set:
            if float(parent[index])<float(thresh):
                left.append(parent)
        return left
    def split_right(self,thresh,index,data_set):#right child
        right=[]
        for parent in data_set:
            if float(parent[index])>=float(thresh):
                right.append(parent)
        return right

    def best_split(self,training_set):#selecting best value to split on using gini impurity and using 20 values between the minimum and maximum possible values.
        min_gin=(np.inf)
        corres_ind=0
        thresh_max=0      
        for i in range(0,len(training_set[0])-1):

            
            thresh=[]
            for j in range(0,len(training_set)):
                thresh.append(float(training_set[j][i]))
            features=np.linspace(min(thresh),max(thresh),30)
            
            for l in range(0,len(features)):
                
                left=self.split_left(float(features[l]),i,training_set)
                right=self.split_right(float(features[l]),i,training_set)
                gin=self.gin(training_set,left,right)
                if float(gin)<float(min_gin):
                    thresh_max=features[l]
                    corres_ind=i

                    min_gin=gin

        return [thresh_max,corres_ind]
            

    tree = {}

    def learn(self, training_set):
        # implement this function
        self.tree = {}
        if self.gini(training_set)!=0 :
            if len(training_set)<=250:#converting into leaf node before the node becomes pure.
              #Pre pruning the decision tree. I observed that the average number of samples in each leaf nodes
              #was around 10. So I decided to use that as the stopping condition .
                
                self.tree['right']=None
                self.tree['left']=None
                self.tree['index']=None
                self.tree['thresh_val']=None
                c0=0
                c1=0
                for instance in training_set:
                    if float(instance[-1])==0:
                        c0=c0+1
                    elif float(instance[-1])==1:
                        c1=c1+1
            #print(c0,c1)
                if c0>=c1:
                    self.tree['value']=0
                elif c1>c0:
                    self.tree['value']=1
                return self.tree        
            self.tree['thresh_val']=(self.best_split(training_set))[0]
            self.tree['index']=(self.best_split(training_set))[1]
            thre=self.tree['thresh_val']
            ind=self.tree['index']
            left_subtree=DecisionTree()#recursively growing the tree
            right_subtree=DecisionTree()
            left=self.split_left(thre,ind,training_set)
            right=self.split_right(thre,ind,training_set)
            left_subtree.learn(left)
            right_subtree.learn(right)
            self.tree['left']=left_subtree.tree
            self.tree['right']=right_subtree.tree
            self.tree['value']=None
        else: # if the node is pure, it will be converted into a leaf node
            self.tree['right']=None
            self.tree['left']=None
            self.tree['index']=None
            self.tree['thresh_val']=None
            c0=0
            c1=0
            for instance in training_set:
                #print(instance)
                
                
                if float(instance[-1])==0:
                    
                    c0=c0+1
                elif float(instance[-1])==1:
                    c1=c1+1
            #print(c0,c1)
            if c0>=c1:
                self.tree['value']=0
            elif c1>c0:
                self.tree['value']=1
        return self.tree        


    # implement this function
    def classify(self, test_instance,x):# we find the value in the dictionary recursively
        #result = 0 # baseline: always classifies as 0
        if (x['value']!=None): 
            
             
            return x['value']
        else:
            
            value_rec=test_instance[x['index']]
            if float(value_rec)<float(x['thresh_val']):
                #print(float(value_rec),float(x['thresh_val']))
                #print(x['left'])
                return self.classify(test_instance,x['left'])
            elif float(value_rec)>=float(x['thresh_val']):
                return self.classify(test_instance,x['right'])

        #return result

def run_decision_tree():
    f=files.upload()
    # Load data set
    with open("wine-dataset.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print("Number of records: %d" % len(data))

    # Split training/test sets
    # You need to modify the following code for cross validation.
    accuracies=[]
    for K in range(0,10):
        
        training_set = [x for i, x in enumerate(data) if i % 10 != K]
        test_set = [x for i, x in enumerate(data) if i % 10== K]
    
        tree = DecisionTree()
        
    # Construct a tree using training set
        dic=tree.learn( training_set)
        #print(dic)
    #print(dic['left']['left']['left']['value'])

    # Classify the test set using the tree we just constructed
        results = []
        for instance in test_set:
            result = tree.classify( instance,dic )
            #print(result)
            results.append( result == float(instance[-1]))
        

    # Accuracy
        accuracy = float(results.count(True))/float(len(results))
        print("accuracy: %.4f" % accuracy)
        accuracies.append(accuracy)       
    sum=0
    for i in range(0,len(accuracies)):
        sum=float(accuracies[i])+sum
    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"result.txt", "w")
    g=sum/len(accuracies)
    f.write("avg. accuracy: %.4f" %g)
    f.close()


if __name__ == "__main__":
    run_decision_tree()

Saving wine-dataset.csv to wine-dataset (1).csv
Number of records: 4898
accuracy: 0.7857
accuracy: 0.8408
accuracy: 0.7980
accuracy: 0.8204
accuracy: 0.8061
accuracy: 0.7959
accuracy: 0.8020
accuracy: 0.7857
accuracy: 0.7669
accuracy: 0.8016
